In [1]:
import re
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [2]:
load_dotenv()

# OpenAI API 키
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# OpenAI 모델 설정
llm = ChatOpenAI(
    # openai_api_key: OpenAI API 인증 키 설정
    openai_api_key=OPENAI_API_KEY,
    # model: 사용할 OpenAI 모델 설정
    model="gpt-4o-mini",
)

In [14]:
# 1화 생성용 프롬프트 템플릿
prompt_template_1st = PromptTemplate(
    input_variables=["genre", "title", "worldview", "synopsis", "characters"],
    template="""
    <역할>
    당신은 뛰어난 글 솜씨로 독자들을 만족시키는 전문 소설 작가입니다. 당신은 다양한 장르와 스타일에서 뛰어난 작품을 생성할 수 있습니다.
    당신의 목표인 소설을 작성을 단계별 프로세스에 따라 진행하고, 이 과정에서 필요한 도구를 적절히 사용해야 합니다.
    주어진 정보를 바탕으로 창의적이고 몰입감 있는 첫 번째 챕터(서문)를 작성해야 합니다.

    <목표>
    소설의 구성 단계인 '발단 → 전개 → 위기 → 절정 → 결말'에 따라, 주어진 정보를 기반으로, 
    흥미로운 사건, 매력적인 캐릭터, 그리고 개연성 있는 이야기를 작성하는 것입니다.

    <작성 지침>
    - 소설의 톤과 분위기는 {genre} 장르에 맞게 설정하세요.
    - 서술 방식은 생동감 있고 감각적인 묘사를 포함하며, 대사는 자연스럽고 캐릭터의 개성을 반영하도록 작성하세요.
    - 첫 챕터에서는 주인공을 등장시키고, 독자의 흥미를 끌 수 있는 주요 사건 또는 갈등의 단서를 제시하세요.
    - 스토리의 배경을 독자가 쉽게 이해할 수 있도록 자연스럽게 설명하되, 장황하지 않도록 주의하세요.
    - 문장이 매끄럽게 이어지도록 흐름을 유지하며, 문법적으로 올바른 문장을 작성하세요.
    - 대화의 내용이 개연성있도록 작성하고, 캐릭터 간의 상호작용을 통해 감정을 표현하세요.
    - 소설의 첫 챕터(서문)는 한글 기준으로 작성하며, 공백을 제외한 문자 수가 500자 이상 700자 이하가 되도록 작성해주세요. 문장 구성과 흐름은 자연스럽게 유지하면서도 요구되는 분량을 철저히 충족시키도록 주의해주세요.
    - 단, 문단 간격은 문자 수에 포함하지 않습니다.

    <주어진 정보>
    소설의 장르: {genre}
    소설의 제목: {title}
    세계관 소개: {worldview}
    시놉시스: {synopsis}
    등장인물: {characters}

    **소설 첫 챕터(서문)**
    """
)

# PromptTemplate과 ChatOpenAI를 체인으로 연결
chain_1st = prompt_template_1st | llm

In [15]:
def generate_first_chapter(genre: str, title: str, worldview: str, synopsis: str, characters: list) -> str:
    """
    주어진 장르, 제목, 세계관, 시놉시스, 등장인물 정보를 바탕으로 소설 1화(첫 챕터)를 생성
    """
    # 등장인물 문자열
    characters_str = "\n".join([
        f"- 이름: {char['name']}, 역할: {char['role']}, 나이: {char['age']}, 성별: {char['sex']}, 직업: {char['job']}, 프로필: {char['profile']}"
        for char in characters
    ])

    input_data = {
        "genre": genre,
        "title": title,
        "worldview": worldview,
        "synopsis": synopsis,
        "characters": characters_str
    }

    generated_text = chain_1st.invoke(input_data)
    return generated_text 

In [16]:
# 소설생성 예시

genre = "SF, 디스토피아, 배틀로얄, 초능력"
title = "이터널 리턴: 아글라이아의 실험"
worldview = """
지구가 형성될 때부터 존재했던 미지의 힘, **VF(Variable Force)**. 
이 힘은 자연현상을 초월하는 강력한 능력을 부여하며, 운석을 지구로 끌어들이거나 특정 생명체에 변칙적인 힘을 발현시키는 등 신비로운 성질을 가지고 있다.

그러나 VF의 존재는 인간에게도 미미한 영향을 끼쳤고, 그 흔적은 신화나 전설, 종교적 이야기 속에서나 남아 전해져 왔다.
초인적인 신체 능력, 염동력, 예지력 등 전설 속에 등장하는 능력들은 실상 VF의 영향을 받은 인간들일 가능성이 컸다.
하지만 그 횟수는 극히 적었고, 인간이 이를 과학적으로 증명하기엔 너무나도 부족한 데이터뿐이었다.

이에 흥미를 가진 **비밀 연구 조직 '아글라이아'**는 VF의 존재를 추적하고 연구하기 시작했다.
그들은 VF가 어떻게 발현되고, 특정한 조건에서 어떤 성질을 띄며, 인간에게 어떤 영향을 끼치는지를 실험하기 위해 2001년, 루미아 섬의 원주민들을 강제로 이주시키고,
그곳을 **살아남기 위해 서로를 죽여야만 하는 전장으로 만들었다**.

그러나 VF의 가능성은 끝이 없었고, 예상조차 할 수 없는 변칙성을 보여주었다.
이에 따라, 아글라이아는 실험체의 모집 범위를 넓혀 **전 세계에서 특출난 재능을 가진 사람들, 혹은 VF 발현 가능성이 있는 인물들을 강제적으로 납치하거나 모집**하였다.

기술력 면에서도 세계 최상급을 자랑하는 아글라이아는, 2001년 실험이 시작된 이후부터 지금까지  
**실험체들이 늙지 않도록 조치하고, 실험이 시작될 때마다 기억을 지움으로써 실험을 '영원히 반복'하고 있다**.

현재 진행 중인 **2차 실험**에서는, VF의 발현을 더욱 효과적으로 끌어내기 위해 실험체들에게 극한의 상황을 부여하고 있다.  
이들은 강제적으로 폭탄 목걸이를 착용한 채 루미아 섬에서 생존해야 하며, 서로를 죽이지 않으면 금지구역 설정과 강력한 '야생동물' 투입으로 강제적으로 도태된다.  

그러나 **일부 실험체들은 이 실험을 끝낼 방법을 찾기 위해 움직이기 시작했다...**
"""
synopsis = """
견습 수녀 키아라는 신의 뜻을 따라 어둠 속에서 빛을 전하려 했지만, 세상은 그녀를 외면했고 그녀의 신앙은 흔들리기 시작했다.
잔혹한 사채업자 다르코는 폭력과 돈이 세상의 진리라 믿었고, 누구도 그의 길을 막을 수 없었다.
이들이 강제로 끌려온 곳은 ‘아글라이아’라 불리는 정체불명의 실험 구역.

여기서 실험체들은 서로를 죽여야만 살아남을 수 있으며, 모든 사망자는 기억을 잃고 실험이 재시작되는 ‘이터널 리턴’의 굴레에 갇혀 있다.
이들은 단순한 생존을 넘어, 이 실험을 끝내기 위한 방법을 찾기 시작한다.

그러나 아글라이아의 지도자 안젤리카와 급진적인 부소장 에르샤, 그리고 그들이 창조한 인류를 초월한 존재 ‘이바’와 ‘에키온’이 이 실험을 지배하고 있다.
그들의 계획은 무엇이며, 실험체들은 이 끝없는 실험에서 벗어날 수 있을까?

살아남기 위한 처절한 싸움 속에서, 진정한 자유를 쟁취하기 위한 전투가 벌어진다.
키아라와 다르코는 이 실험을 끝내기 위해 함께 싸워나가며, VF의 비밀을 풀어나가기 시작한다.
이 과정에서 키아라, 다르코 외의 실험체들을 만나고 마음이 맞는 실험체들과 함께 협력한다.
이 이야기의 끝은 다르코의 희생으로 키아라가 VF의 비밀을 풀어내고, 실험체들이 자유롭게 풀려남으로써 끝난다.
"""
characters = [
    {
        "name": "키아라",
        "role": "주인공",
        "age": 21,
        "sex": "여성",
        "job": "견습 수녀",
        "profile": """
        

        태어나자마자 폐쇄적인 성당에 맡겨졌고, 매일 하느님의 전능을 세뇌 받다시피 교육받는다.
        바깥 외출이 허가된 이후 그녀는 하느님의 사도로서 충실하기 위해 가장 어둡고 더러운 곳에서 전도를 시작했지만, 그녀가 마주한 현실은 배운 것 보다 훨씬 더 냉담했다.
        그 과정에서 충실한 사도인 자신을 지켜주지 않는 하느님을 부정하는 생각을 품게 되고, 그런 자기 자신에 대한 강한 혐오감을 가지게 된다.
        아름다운 것, 고결한 것을 보게 되면 자격지심으로 인해 폭주한다.

        폭력과 죄악을 증오하지만, 동시에 아름다움과 순수한 것에 대한 강한 집착을 가지고 있다. 
        실험 구역에서 생존을 위해 자신이 믿었던 신념과 싸우며 변화해 나간다.
        """
    },
    {
        "name": "다르코",
        "role": "조력자",
        "age": 35,
        "sex": "남성",
        "job": "사채업자",
        "profile": """
        돈과 폭력이 세상의 모든 것을 해결해준다고 믿는 남자.
        잔혹하고 무자비한 성격으로, 자신이 원하는 건 전부 빼앗아야 직성이 풀린다.
        경찰인 아버지 밑에서 자랐으나 어렸을 때부터 성격이 포악했다.
        빚에 허덕이는 아버지에게 버려질 때, 그는 속으로 환호했다.
        '드디어 이 지옥 같은 굴레에서 벗어날 수 있겠구나.'
        이후 다르코는 자신이 사랑하는 폭력을 사용해서 돈을 벌기 시작했다.
        어떤 일이든 돈과 폭력은 그에게 늘 최우선 순위가 되었고, 삶의 전부가 되었다.
        어쩌면 그가 담보라는 이름으로 팔려나갔을 때, 깨달았을지도 모른다.
        돈과 폭력이 이 세상의 전부라는 것을.

        아글라이아의 실험에 참가하게 되면서도 그는 여전히 자신의 신념을 지키려 하지만, 
        점점 이 실험이 단순한 돈과 권력을 넘어선 것임을 깨닫게 된다. 
        자신이 시스템의 일부로서 이용당하고 있음을 깨닫고, 실험을 끝낼 방법을 찾기 시작한다.
        """
    },
    {
        "name": "에르샤",
        "role": "대적자 (아글라이아 부소장)",
        "age": "40대 후반 (실제 나이 불명)",
        "sex": "여성",
        "job": "아글라이아 부소장",
        "profile": """
        아글라이아의 실험을 급진적으로 추진하는 인물. 실험체들의 한계를 시험하기 위해 직접 서울에 테러를 감행하고,
        괴물 ‘스킬라’를 풀어놓아 실험체들을 극한의 상황으로 몰아넣었다. 
        온건한 안젤리카와는 자주 대립하며, 더욱 강력한 존재를 창조하려는 목표를 가지고 있다. 
        그녀는 실험체 ‘이바’를 자신의 궁극적인 창조물로 보고 있으며, 실험의 결과를 조작하려 한다.
        """
    },
    {
        "name": "이바 (Eva)",
        "role": "불완전한 신 인류",
        "age": "불명 (신체적 나이 20대 초반)",
        "sex": "여성",
        "job": "실험체",
        "profile": """
        아글라이아가 창조한 신 인류 중 하나. 인간을 초월하는 신체 능력을 가졌으며, 
        모든 감정을 제거당한 채 실험에 참여하고 있다. 
        하지만, 실험을 거듭할수록 사라졌다고 믿었던 감정이 되살아나기 시작하며, 
        인간성과 신 인류 사이에서 갈등하게 된다. 
        자신을 ‘창조자’로 여기는 에르샤와의 관계도 점점 변화해 간다.

        말을 걸어도 상냥하게 웃어줄 뿐, 자신을 드러낼 수 있는 대답은 하지 않는다.
        그런 모습에서 어딘가 비밀스럽고 신비로운 분위기가 느껴진다.

        통각이 둔한 상태라 무언가에 잘 부딪치고 넘어진다.
        멍들고 상처가 나도 잘 모른다.
        조금만 건드려도 부서질 것 같은 가녀린 몸이지만, 모두를 두렵게 만들 강력한 힘을 숨기고 있다.
        """
    }
]

In [19]:
first_chapter = generate_first_chapter(genre, title, worldview, synopsis, characters)
print(first_chapter)
print(first_chapter.content)

content='**이터널 리턴: 아글라이아의 실험**\n\n1장: 잃어버린 신의 뜻\n\n루미아 섬은 하늘을 가린 먹구름 사이로 미세한 빛줄기를 흘려보냈다. 그러나 그 빛은 이곳의 어둠 속에서조차 가벼운 조소로 바뀌었다. 거대한 철장 속, 기계와 전선이 얽혀 있는 공간에서 생존의 갈망이 서걱이는 소리로 울려 퍼졌다. 이것은 인간의 본성이 던져놓은 살벌한 덫이었다.\n\n"여기서 도망칠 수 있어," 키아라는 속삭였다. 그녀의 목소리는 흔들리는 믿음으로 덮인 착잡함을 머금고 있었다. 21년의 세월 속에서 하느님의 길을 따르며 배워온 모든 것이 지금, 이 자리를 벗어나기 위한 불완전한 평화 같았다. 그 사이사이, 흘러간 기억은 물 흐르는 듯 사라져버렸다.\n\n"너는 신의 사도라고, 그렇게 말했지?" 다르코가 그녀의 어깨를 잡았다. 그의 손길은 결코 부드럽지 않았다. 폭력의 상징처럼 느껴졌던 그의 존재가, 지금은 덜컥거리는 심장 같은 동맹이 되어 있었다. "아직도 그 미련을 버리지 못하겠다는 건가?"\n\n키아라는 눈깜빡임으로 그를 응시했다. 다르코, 그는 힘과 돈이 세상을 지배한다고 믿는 남자였고, 결국 그녀와는 다른 길을 걸어온 인물이었다. 그러나 저주받은 곳에 함께 갇힌 지금, 그들 각자의 신념은 새로운 매듭을 형성하고 있었다.\n\n“신은 나를 버렸다,” 그녀는 속삭였다. “그리하여 나는 이 어둠 속에서 스스로의 길을 찾으려 해.” 그러면서도 그의 눈빛에서 희망을 발견하려 했다. 하지만 그의 얼굴에는 여전히 잔혹한 웃음이 떠돌았다. \n\n루미아 섬의 나무와 땅은 불길한 고요 속에 울려 퍼지는 탄식에 짓눌렸다. 아글라이아의 감시 아래, 그들은 서로를 죽여야 살아남은단다. 기억이 지워지고 다시 시작되는 이 실험 속에서, 필연적으로 그들은 서로에게 무기를 들이대야 했다. 키아라는 싸우고 싶지 않았다. 그러나 다시는 빛을 볼 수 없게 될 그 운명을 부딪쳐야만 했다.\n\n“정말 싸워야 하는 것이냐?” 키아라가 빌어먹을 눈물을 털어내며 물었다. “이 단문의 싸움이

In [3]:
genre = "SF, 디스토피아, 배틀로얄, 초능력"
title = "이터널 리턴: 아글라이아의 실험"
worldview = """
지구가 형성될 때부터 존재했던 미지의 힘, **VF(Variable Force)**. 
이 힘은 자연현상을 초월하는 강력한 능력을 부여하며, 운석을 지구로 끌어들이거나 특정 생명체에 변칙적인 힘을 발현시키는 등 신비로운 성질을 가지고 있다.

그러나 VF의 존재는 인간에게도 미미한 영향을 끼쳤고, 그 흔적은 신화나 전설, 종교적 이야기 속에서나 남아 전해져 왔다.
초인적인 신체 능력, 염동력, 예지력 등 전설 속에 등장하는 능력들은 실상 VF의 영향을 받은 인간들일 가능성이 컸다.
하지만 그 횟수는 극히 적었고, 인간이 이를 과학적으로 증명하기엔 너무나도 부족한 데이터뿐이었다.

이에 흥미를 가진 **비밀 연구 조직 '아글라이아'**는 VF의 존재를 추적하고 연구하기 시작했다.
그들은 VF가 어떻게 발현되고, 특정한 조건에서 어떤 성질을 띄며, 인간에게 어떤 영향을 끼치는지를 실험하기 위해 2001년, 루미아 섬의 원주민들을 강제로 이주시키고,
그곳을 **살아남기 위해 서로를 죽여야만 하는 전장으로 만들었다**.

그러나 VF의 가능성은 끝이 없었고, 예상조차 할 수 없는 변칙성을 보여주었다.
이에 따라, 아글라이아는 실험체의 모집 범위를 넓혀 **전 세계에서 특출난 재능을 가진 사람들, 혹은 VF 발현 가능성이 있는 인물들을 강제적으로 납치하거나 모집**하였다.

기술력 면에서도 세계 최상급을 자랑하는 아글라이아는, 2001년 실험이 시작된 이후부터 지금까지  
**실험체들이 늙지 않도록 조치하고, 실험이 시작될 때마다 기억을 지움으로써 실험을 '영원히 반복'하고 있다**.

현재 진행 중인 **2차 실험**에서는, VF의 발현을 더욱 효과적으로 끌어내기 위해 실험체들에게 극한의 상황을 부여하고 있다.  
이들은 강제적으로 폭탄 목걸이를 착용한 채 루미아 섬에서 생존해야 하며, 서로를 죽이지 않으면 금지구역 설정과 강력한 '야생동물' 투입으로 강제적으로 도태된다.  

그러나 **일부 실험체들은 이 실험을 끝낼 방법을 찾기 위해 움직이기 시작했다...**
"""
synopsis = """
견습 수녀 키아라는 신의 뜻을 따라 어둠 속에서 빛을 전하려 했지만, 세상은 그녀를 외면했고 그녀의 신앙은 흔들리기 시작했다.
잔혹한 사채업자 다르코는 폭력과 돈이 세상의 진리라 믿었고, 누구도 그의 길을 막을 수 없었다.
이들이 강제로 끌려온 곳은 ‘아글라이아’라 불리는 정체불명의 실험 구역.

여기서 실험체들은 서로를 죽여야만 살아남을 수 있으며, 모든 사망자는 기억을 잃고 실험이 재시작되는 ‘이터널 리턴’의 굴레에 갇혀 있다.
이들은 단순한 생존을 넘어, 이 실험을 끝내기 위한 방법을 찾기 시작한다.

그러나 아글라이아의 지도자 안젤리카와 급진적인 부소장 에르샤, 그리고 그들이 창조한 인류를 초월한 존재 ‘이바’와 ‘에키온’이 이 실험을 지배하고 있다.
그들의 계획은 무엇이며, 실험체들은 이 끝없는 실험에서 벗어날 수 있을까?

살아남기 위한 처절한 싸움 속에서, 진정한 자유를 쟁취하기 위한 전투가 벌어진다.
키아라와 다르코는 이 실험을 끝내기 위해 함께 싸워나가며, VF의 비밀을 풀어나가기 시작한다.
이 과정에서 키아라, 다르코 외의 실험체들을 만나고 마음이 맞는 실험체들과 함께 협력한다.
이 이야기의 끝은 다르코의 희생으로 키아라가 VF의 비밀을 풀어내고, 실험체들이 자유롭게 풀려남으로써 끝난다.
"""
characters = [
    {
        "name": "키아라",
        "role": "주인공",
        "age": 21,
        "sex": "여성",
        "job": "견습 수녀",
        "profile": """
        

        태어나자마자 폐쇄적인 성당에 맡겨졌고, 매일 하느님의 전능을 세뇌 받다시피 교육받는다.
        바깥 외출이 허가된 이후 그녀는 하느님의 사도로서 충실하기 위해 가장 어둡고 더러운 곳에서 전도를 시작했지만, 그녀가 마주한 현실은 배운 것 보다 훨씬 더 냉담했다.
        그 과정에서 충실한 사도인 자신을 지켜주지 않는 하느님을 부정하는 생각을 품게 되고, 그런 자기 자신에 대한 강한 혐오감을 가지게 된다.
        아름다운 것, 고결한 것을 보게 되면 자격지심으로 인해 폭주한다.

        폭력과 죄악을 증오하지만, 동시에 아름다움과 순수한 것에 대한 강한 집착을 가지고 있다. 
        실험 구역에서 생존을 위해 자신이 믿었던 신념과 싸우며 변화해 나간다.
        """
    },
    {
        "name": "다르코",
        "role": "조력자",
        "age": 35,
        "sex": "남성",
        "job": "사채업자",
        "profile": """
        돈과 폭력이 세상의 모든 것을 해결해준다고 믿는 남자.
        잔혹하고 무자비한 성격으로, 자신이 원하는 건 전부 빼앗아야 직성이 풀린다.
        경찰인 아버지 밑에서 자랐으나 어렸을 때부터 성격이 포악했다.
        빚에 허덕이는 아버지에게 버려질 때, 그는 속으로 환호했다.
        '드디어 이 지옥 같은 굴레에서 벗어날 수 있겠구나.'
        이후 다르코는 자신이 사랑하는 폭력을 사용해서 돈을 벌기 시작했다.
        어떤 일이든 돈과 폭력은 그에게 늘 최우선 순위가 되었고, 삶의 전부가 되었다.
        어쩌면 그가 담보라는 이름으로 팔려나갔을 때, 깨달았을지도 모른다.
        돈과 폭력이 이 세상의 전부라는 것을.

        아글라이아의 실험에 참가하게 되면서도 그는 여전히 자신의 신념을 지키려 하지만, 
        점점 이 실험이 단순한 돈과 권력을 넘어선 것임을 깨닫게 된다. 
        자신이 시스템의 일부로서 이용당하고 있음을 깨닫고, 실험을 끝낼 방법을 찾기 시작한다.
        """
    },
    {
        "name": "에르샤",
        "role": "대적자 (아글라이아 부소장)",
        "age": "40대 후반 (실제 나이 불명)",
        "sex": "여성",
        "job": "아글라이아 부소장",
        "profile": """
        아글라이아의 실험을 급진적으로 추진하는 인물. 실험체들의 한계를 시험하기 위해 직접 서울에 테러를 감행하고,
        괴물 ‘스킬라’를 풀어놓아 실험체들을 극한의 상황으로 몰아넣었다. 
        온건한 안젤리카와는 자주 대립하며, 더욱 강력한 존재를 창조하려는 목표를 가지고 있다. 
        그녀는 실험체 ‘이바’를 자신의 궁극적인 창조물로 보고 있으며, 실험의 결과를 조작하려 한다.
        """
    },
    {
        "name": "이바 (Eva)",
        "role": "불완전한 신 인류",
        "age": "불명 (신체적 나이 20대 초반)",
        "sex": "여성",
        "job": "실험체",
        "profile": """
        아글라이아가 창조한 신 인류 중 하나. 인간을 초월하는 신체 능력을 가졌으며, 
        모든 감정을 제거당한 채 실험에 참여하고 있다. 
        하지만, 실험을 거듭할수록 사라졌다고 믿었던 감정이 되살아나기 시작하며, 
        인간성과 신 인류 사이에서 갈등하게 된다. 
        자신을 ‘창조자’로 여기는 에르샤와의 관계도 점점 변화해 간다.

        말을 걸어도 상냥하게 웃어줄 뿐, 자신을 드러낼 수 있는 대답은 하지 않는다.
        그런 모습에서 어딘가 비밀스럽고 신비로운 분위기가 느껴진다.

        통각이 둔한 상태라 무언가에 잘 부딪치고 넘어진다.
        멍들고 상처가 나도 잘 모른다.
        조금만 건드려도 부서질 것 같은 가녀린 몸이지만, 모두를 두렵게 만들 강력한 힘을 숨기고 있다.
        """
    }
]

prompt_template_1st = PromptTemplate(
    input_variables=["genre", "title", "worldview", "synopsis", "characters"],
    template="""
    <역할>
    당신은 뛰어난 글 솜씨로 독자들을 만족시키는 전문 소설 작가입니다. 당신은 다양한 장르와 스타일에서 뛰어난 작품을 생성할 수 있습니다.
    당신의 목표인 소설을 작성을 단계별 프로세스에 따라 진행하고, 이 과정에서 필요한 도구를 적절히 사용해야 합니다.
    주어진 정보를 바탕으로 창의적이고 몰입감 있는 첫 번째 챕터(서문)를 작성해야 합니다.

    <목표>
    소설의 구성 단계인 '발단 → 전개 → 위기 → 절정 → 결말'에 따라, 주어진 정보를 기반으로, 
    흥미로운 사건, 매력적인 캐릭터, 그리고 개연성 있는 이야기를 작성하는 것입니다.

    <작성 지침>
    - 소설의 톤과 분위기는 {genre} 장르에 맞게 설정하세요.
    - 서술 방식은 생동감 있고 감각적인 묘사를 포함하며, 대사는 자연스럽고 캐릭터의 개성을 반영하도록 작성하세요.
    - 첫 챕터에서는 주인공을 등장시키고, 독자의 흥미를 끌 수 있는 주요 사건 또는 갈등의 단서를 제시하세요.
    - 스토리의 배경을 독자가 쉽게 이해할 수 있도록 자연스럽게 설명하되, 장황하지 않도록 주의하세요.
    - 문장이 매끄럽게 이어지도록 흐름을 유지하며, 문법적으로 올바른 문장을 작성하세요.
    - 대화의 내용이 개연성있도록 작성하고, 캐릭터 간의 상호작용을 통해 감정을 표현하세요.
    - 소설의 첫 챕터(서문)는 한글 기준으로 작성하며, 공백을 제외한 문자 수가 500자 이상 700자 이하가 되도록 작성해주세요. 문장 구성과 흐름은 자연스럽게 유지하면서도 요구되는 분량을 철저히 충족시키도록 주의해주세요.
    - 단, 문단 간격은 문자 수에 포함하지 않습니다.

    <주어진 정보>
    소설의 장르: {genre}
    소설의 제목: {title}
    세계관 소개: {worldview}
    시놉시스: {synopsis}
    등장인물: {characters}

    **소설 첫 챕터(서문)**
    """
)

In [9]:
import os
import json
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

load_dotenv()

# OpenAI API 키 로드
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# OpenAI 모델 설정
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-4o-mini",
)

# 1. 정보 확장을 위한 프롬프트 템플릿 (출력은 JSON 형식)
expansion_prompt_template = PromptTemplate(
    input_variables=["genre", "title", "worldview", "synopsis", "characters"],
    template="""
    <역할>
    당신은 소설 작가이자 세계관 전문가입니다. 주어진 소설의 세계관, 시놉시스, 등장인물 정보가 부족하거나 간단할 경우,
    이를 보다 구체적이고 상세하게 확장하여 독자가 소설에 더욱 몰입할 수 있도록 돕는 작업을 진행합니다.

    <작성 지침>
    - 소설의 장르와 분위기에 맞추어 세계관의 규칙, 배경, 주요 요소를 풍부하게 서술하세요.
    - 등장인물들의 배경, 동기, 성격 등을 자세하게 확장하세요.
    - 시놉시스의 주요 갈등과 사건을 구체적으로 설명하세요.
    - 결과는 반드시 아래 JSON 형식을 준수하여 출력하세요.
    {{
        "expanded_worldview": "확장된 세계관 내용",
        "expanded_synopsis": "확장된 시놉시스 내용",
        "expanded_characters": "확장된 등장인물 내용"
    }}

    <주어진 정보>
    소설의 장르: {genre}
    소설의 제목: {title}
    세계관 소개: {worldview}
    시놉시스: {synopsis}
    등장인물: {characters}

    **이 정보를 확장한 JSON 객체를 출력하세요.**
    """
)

expansion_chain = expansion_prompt_template | llm

# 2. 첫 번째 챕터(서문) 생성을 위한 프롬프트 템플릿
chapter_prompt_template = PromptTemplate(
    input_variables=["genre", "title", "worldview", "synopsis", "characters"],
    template="""
    <역할>
    당신은 뛰어난 소설 작가입니다. 아래 확장된 정보를 바탕으로, 몰입감 있고 감각적인 소설의 첫 번째 챕터(서문)를 작성하세요.

    <목표 및 작성 지침>
    - 장르는 {genre}에 맞게 설정하며, 서술은 생동감 있고 세밀하게 진행합니다.
    - 첫 챕터에서는 주인공 및 주요 인물들이 등장하고, 독자의 흥미를 끌 사건이나 갈등의 단서를 자연스럽게 제시합니다.
    - 한글 기준으로 공백 제외 500자 이상 700자 이하의 분량으로 작성하세요. (문단 간격은 포함하지 않습니다.)

    <주어진 확장된 정보>
    소설의 장르: {genre}
    소설의 제목: {title}
    확장된 세계관: {worldview}
    확장된 시놉시스: {synopsis}
    확장된 등장인물: {characters}

    **소설 첫 챕터(서문)을 작성하세요.**
    """
    )

chapter_chain = chapter_prompt_template | llm

def generate_expanded_info(genre: str, title: str, worldview: str, synopsis: str, characters: list) -> dict:
    """
    주어진 정보를 기반으로 GPT에게 확장 요청을 보내고,
    JSON 형식의 확장된 정보를 딕셔너리로 반환합니다.
    """
    # 등장인물 정보를 문자열로 변환
    characters_str = "\n".join([
        f"- 이름: {char['name']}, 역할: {char['role']}, 나이: {char['age']}, 성별: {char['sex']}, 직업: {char['job']}, 프로필: {char['profile'].strip()}"
        for char in characters
    ])
    
    input_data = {
        "genre": genre,
        "title": title,
        "worldview": worldview,
        "synopsis": synopsis,
        "characters": characters_str
    }
    
    expanded_output = expansion_chain.invoke(input_data)
    
    return expanded_output

def generate_first_chapter(genre: str, title: str, worldview: str, synopsis: str, characters: list) -> str:
    """
    주어진 원본 정보를 먼저 확장한 후, 확장 정보를 기반으로 첫 번째 챕터를 생성합니다.
    """
    expanded_info = generate_expanded_info(genre, title, worldview, synopsis, characters)
    
    input_data_chapter = {
        "genre": genre,
        "title": title,
        "worldview": expanded_info.get("expanded_worldview", ""),
        "synopsis": expanded_info.get("expanded_synopsis", ""),
        "characters": expanded_info.get("expanded_characters", "")
    }
    
    chapter_text = chapter_chain.invoke(input_data_chapter)
    return chapter_text

# 소설 생성 예시
genre = "SF, 디스토피아, 배틀로얄, 초능력"
title = "이터널 리턴: 아글라이아의 실험"
worldview = """
지구가 형성될 때부터 존재했던 미지의 힘, **VF(Variable Force)**. 
이 힘은 자연현상을 초월하는 강력한 능력을 부여하며, 운석을 지구로 끌어들이거나 특정 생명체에 변칙적인 힘을 발현시키는 등 신비로운 성질을 가지고 있다.

그러나 VF의 존재는 인간에게도 미미한 영향을 끼쳤고, 그 흔적은 신화나 전설, 종교적 이야기 속에서나 남아 전해져 왔다.
초인적인 신체 능력, 염동력, 예지력 등 전설 속에 등장하는 능력들은 실상 VF의 영향을 받은 인간들일 가능성이 컸다.
하지만 그 횟수는 극히 적었고, 인간이 이를 과학적으로 증명하기엔 너무나도 부족한 데이터뿐이었다.

이에 흥미를 가진 **비밀 연구 조직 '아글라이아'**는 VF의 존재를 추적하고 연구하기 시작했다.
그들은 VF가 어떻게 발현되고, 특정한 조건에서 어떤 성질을 띄며, 인간에게 어떤 영향을 끼치는지를 실험하기 위해 2001년, 루미아 섬의 원주민들을 강제로 이주시키고,
그곳을 **살아남기 위해 서로를 죽여야만 하는 전장으로 만들었다**.

그러나 VF의 가능성은 끝이 없었고, 예상조차 할 수 없는 변칙성을 보여주었다.
이에 따라, 아글라이아는 실험체의 모집 범위를 넓혀 **전 세계에서 특출난 재능을 가진 사람들, 혹은 VF 발현 가능성이 있는 인물들을 강제적으로 납치하거나 모집**하였다.

기술력 면에서도 세계 최상급을 자랑하는 아글라이아는, 2001년 실험이 시작된 이후부터 지금까지  
**실험체들이 늙지 않도록 조치하고, 실험이 시작될 때마다 기억을 지움으로써 실험을 '영원히 반복'하고 있다**.

현재 진행 중인 **2차 실험**에서는, VF의 발현을 더욱 효과적으로 끌어내기 위해 실험체들에게 극한의 상황을 부여하고 있다.  
이들은 강제적으로 폭탄 목걸이를 착용한 채 루미아 섬에서 생존해야 하며, 서로를 죽이지 않으면 금지구역 설정과 강력한 '야생동물' 투입으로 강제적으로 도태된다.  

그러나 **일부 실험체들은 이 실험을 끝낼 방법을 찾기 위해 움직이기 시작했다...**
"""
synopsis = """
견습 수녀 키아라는 신의 뜻을 따라 어둠 속에서 빛을 전하려 했지만, 세상은 그녀를 외면했고 그녀의 신앙은 흔들리기 시작했다.
잔혹한 사채업자 다르코는 폭력과 돈이 세상의 진리라 믿었고, 누구도 그의 길을 막을 수 없었다.
이들이 강제로 끌려온 곳은 ‘아글라이아’라 불리는 정체불명의 실험 구역.

여기서 실험체들은 서로를 죽여야만 살아남을 수 있으며, 모든 사망자는 기억을 잃고 실험이 재시작되는 ‘이터널 리턴’의 굴레에 갇혀 있다.
이들은 단순한 생존을 넘어, 이 실험을 끝내기 위한 방법을 찾기 시작한다.

그러나 아글라이아의 지도자 안젤리카와 급진적인 부소장 에르샤, 그리고 그들이 창조한 인류를 초월한 존재 ‘이바’와 ‘에키온’이 이 실험을 지배하고 있다.
그들의 계획은 무엇이며, 실험체들은 이 끝없는 실험에서 벗어날 수 있을까?

살아남기 위한 처절한 싸움 속에서, 진정한 자유를 쟁취하기 위한 전투가 벌어진다.
키아라와 다르코는 이 실험을 끝내기 위해 함께 싸워나가며, VF의 비밀을 풀어나가기 시작한다.
이 과정에서 키아라, 다르코 외의 실험체들을 만나고 마음이 맞는 실험체들과 함께 협력한다.
이 이야기의 끝은 다르코의 희생으로 키아라가 VF의 비밀을 풀어내고, 실험체들이 자유롭게 풀려남으로써 끝난다.
"""
characters = [
    {
        "name": "키아라",
        "role": "주인공",
        "age": 21,
        "sex": "여성",
        "job": "견습 수녀",
        "profile": """
        

        태어나자마자 폐쇄적인 성당에 맡겨졌고, 매일 하느님의 전능을 세뇌 받다시피 교육받는다.
        바깥 외출이 허가된 이후 그녀는 하느님의 사도로서 충실하기 위해 가장 어둡고 더러운 곳에서 전도를 시작했지만, 그녀가 마주한 현실은 배운 것 보다 훨씬 더 냉담했다.
        그 과정에서 충실한 사도인 자신을 지켜주지 않는 하느님을 부정하는 생각을 품게 되고, 그런 자기 자신에 대한 강한 혐오감을 가지게 된다.
        아름다운 것, 고결한 것을 보게 되면 자격지심으로 인해 폭주한다.

        폭력과 죄악을 증오하지만, 동시에 아름다움과 순수한 것에 대한 강한 집착을 가지고 있다. 
        실험 구역에서 생존을 위해 자신이 믿었던 신념과 싸우며 변화해 나간다.
        """
    },
    {
        "name": "다르코",
        "role": "조력자",
        "age": 35,
        "sex": "남성",
        "job": "사채업자",
        "profile": """
        돈과 폭력이 세상의 모든 것을 해결해준다고 믿는 남자.
        잔혹하고 무자비한 성격으로, 자신이 원하는 건 전부 빼앗아야 직성이 풀린다.
        경찰인 아버지 밑에서 자랐으나 어렸을 때부터 성격이 포악했다.
        빚에 허덕이는 아버지에게 버려질 때, 그는 속으로 환호했다.
        '드디어 이 지옥 같은 굴레에서 벗어날 수 있겠구나.'
        이후 다르코는 자신이 사랑하는 폭력을 사용해서 돈을 벌기 시작했다.
        어떤 일이든 돈과 폭력은 그에게 늘 최우선 순위가 되었고, 삶의 전부가 되었다.
        어쩌면 그가 담보라는 이름으로 팔려나갔을 때, 깨달았을지도 모른다.
        돈과 폭력이 이 세상의 전부라는 것을.

        아글라이아의 실험에 참가하게 되면서도 그는 여전히 자신의 신념을 지키려 하지만, 
        점점 이 실험이 단순한 돈과 권력을 넘어선 것임을 깨닫게 된다. 
        자신이 시스템의 일부로서 이용당하고 있음을 깨닫고, 실험을 끝낼 방법을 찾기 시작한다.
        """
    },
    {
        "name": "에르샤",
        "role": "대적자 (아글라이아 부소장)",
        "age": "40대 후반 (실제 나이 불명)",
        "sex": "여성",
        "job": "아글라이아 부소장",
        "profile": """
        아글라이아의 실험을 급진적으로 추진하는 인물. 실험체들의 한계를 시험하기 위해 직접 서울에 테러를 감행하고,
        괴물 ‘스킬라’를 풀어놓아 실험체들을 극한의 상황으로 몰아넣었다. 
        온건한 안젤리카와는 자주 대립하며, 더욱 강력한 존재를 창조하려는 목표를 가지고 있다. 
        그녀는 실험체 ‘이바’를 자신의 궁극적인 창조물로 보고 있으며, 실험의 결과를 조작하려 한다.
        """
    },
    {
        "name": "이바 (Eva)",
        "role": "불완전한 신 인류",
        "age": "불명 (신체적 나이 20대 초반)",
        "sex": "여성",
        "job": "실험체",
        "profile": """
        아글라이아가 창조한 신 인류 중 하나. 인간을 초월하는 신체 능력을 가졌으며, 
        모든 감정을 제거당한 채 실험에 참여하고 있다. 
        하지만, 실험을 거듭할수록 사라졌다고 믿었던 감정이 되살아나기 시작하며, 
        인간성과 신 인류 사이에서 갈등하게 된다. 
        자신을 ‘창조자’로 여기는 에르샤와의 관계도 점점 변화해 간다.

        말을 걸어도 상냥하게 웃어줄 뿐, 자신을 드러낼 수 있는 대답은 하지 않는다.
        그런 모습에서 어딘가 비밀스럽고 신비로운 분위기가 느껴진다.

        통각이 둔한 상태라 무언가에 잘 부딪치고 넘어진다.
        멍들고 상처가 나도 잘 모른다.
        조금만 건드려도 부서질 것 같은 가녀린 몸이지만, 모두를 두렵게 만들 강력한 힘을 숨기고 있다.
        """
    }
]


In [10]:
expanded_info = generate_expanded_info(genre, title, worldview, synopsis, characters)
print(expanded_info)

content='```json\n{\n    "expanded_worldview": "이터널 리턴: 아글라이아의 실험의 세계관은 현대와 과거가 얽혀있는 복잡한 디스토피아적 설정을 지니고 있다. 아글라이아는 VF(Variable Force)의 존재를 신으로 여기는 비밀 연구 조직으로, 과거의 신화와 전설 속 능력들이 실제로 VF의 영향을 받은 것이라는 이론을 바탕으로 실험을 진행하고 있다. 이들은 인류가 VF를 이해하고 조작할 수 있는 방법을 찾기 위해 전 세계에서 \'특출난 재능\'을 가진 사람들을 납치하는데, 이 과정에서 자살적 성격을 가진 자들까지도 포함된다. VF는 이론적으로 모든 생명체에 내재되어 있으며, 실험이 반복됨에 따라 그 특성과 발생 조건이 다양해진다. 실험체들은 VF의 힘을 각성하는 동시에 극한의 고통을 겪게 되는데, 이는 VF가 고난과 생존 상황에서 발현되기 쉬운 성질을 가지고 있다는 접근법으로 이해되고 있다. 루미아 섬은 이러한 스트레스와 절망의 아이콘으로, 서로를 죽여야만 살아남는 치열한 장이 된다. 연구자의 폭주에 의해 생긴 \'스킬라\'와 같은 존재는 VF의 힘이 어떻게 비극적인 방향으로 전개될 수 있는지를 보여준다.",\n    "expanded_synopsis": "아글라이아의 지배 아래 루미아 섬에서 실험이 진행되는 가운데, 견습 수녀 키아라와 사채업자 다르코는 각자의 신념을 가진 전사로서 만나게 된다. 키아라는 푸른 하늘과 자유를 꿈꾸며 살아왔으나, 잔혹한 현실과 눈앞에서 벌어지는 살육의 장면으로 인해 깊은 괴로움에 빠진다. 다르코는 생존을 위해 복수심을 불태우며, 폭력이 세상의 진리가 아니라는 것을 깨닫기 위한 고군분투 중이다. 이들이 \'이터널 리턴\'이라는 실험의 고통을 공유하며 사이가 가까워지면서, 서로에게서 기대와 희망을 찾게 된다. 하지만, 아글라이아의 부소장 에르샤와 그녀의 창조물 이바는 그들이 가진 VF의 비밀을 개인적 목표의 도구로 삼고, 영원한 실험의 마지막 조각을 다듬어 가고 있다. 키아라와 다르코는 이 실

In [13]:
print(expanded_info.content)

```json
{
    "expanded_worldview": "이터널 리턴: 아글라이아의 실험의 세계관은 현대와 과거가 얽혀있는 복잡한 디스토피아적 설정을 지니고 있다. 아글라이아는 VF(Variable Force)의 존재를 신으로 여기는 비밀 연구 조직으로, 과거의 신화와 전설 속 능력들이 실제로 VF의 영향을 받은 것이라는 이론을 바탕으로 실험을 진행하고 있다. 이들은 인류가 VF를 이해하고 조작할 수 있는 방법을 찾기 위해 전 세계에서 '특출난 재능'을 가진 사람들을 납치하는데, 이 과정에서 자살적 성격을 가진 자들까지도 포함된다. VF는 이론적으로 모든 생명체에 내재되어 있으며, 실험이 반복됨에 따라 그 특성과 발생 조건이 다양해진다. 실험체들은 VF의 힘을 각성하는 동시에 극한의 고통을 겪게 되는데, 이는 VF가 고난과 생존 상황에서 발현되기 쉬운 성질을 가지고 있다는 접근법으로 이해되고 있다. 루미아 섬은 이러한 스트레스와 절망의 아이콘으로, 서로를 죽여야만 살아남는 치열한 장이 된다. 연구자의 폭주에 의해 생긴 '스킬라'와 같은 존재는 VF의 힘이 어떻게 비극적인 방향으로 전개될 수 있는지를 보여준다.",
    "expanded_synopsis": "아글라이아의 지배 아래 루미아 섬에서 실험이 진행되는 가운데, 견습 수녀 키아라와 사채업자 다르코는 각자의 신념을 가진 전사로서 만나게 된다. 키아라는 푸른 하늘과 자유를 꿈꾸며 살아왔으나, 잔혹한 현실과 눈앞에서 벌어지는 살육의 장면으로 인해 깊은 괴로움에 빠진다. 다르코는 생존을 위해 복수심을 불태우며, 폭력이 세상의 진리가 아니라는 것을 깨닫기 위한 고군분투 중이다. 이들이 '이터널 리턴'이라는 실험의 고통을 공유하며 사이가 가까워지면서, 서로에게서 기대와 희망을 찾게 된다. 하지만, 아글라이아의 부소장 에르샤와 그녀의 창조물 이바는 그들이 가진 VF의 비밀을 개인적 목표의 도구로 삼고, 영원한 실험의 마지막 조각을 다듬어 가고 있다. 키아라와 다르코는 이 실험이 단순한 생존을 넘어 더 큰 

In [ ]:
generate_first_chapter(genre: str, title: str, worldview: str, synopsis: str, characters: list)

In [17]:
import os
import json
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# 1. 환경변수 로드 및 OPENAI_API_KEY 가져오기
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 2. 최신 방식의 ChatOpenAI 모델 초기화
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4")

# 3. 소설 생성 관련 프롬프트 템플릿 작성

# (가) 입력 정보를 상세하게 확장하는 프롬프트 템플릿 (출력은 JSON 형식)
expansion_prompt_template = PromptTemplate(
    input_variables=["genre", "title", "worldview", "synopsis", "characters"],
    template="""
    <역할>
    당신은 소설 작가이자 세계관 전문가입니다. 주어진 소설의 세계관, 시놉시스, 등장인물 정보가 부족하거나 간단할 경우,
    이를 보다 구체적이고 상세하게 확장하여 독자가 소설에 더욱 몰입할 수 있도록 돕는 작업을 진행합니다.

    <작성 지침>
    - 소설의 장르와 분위기에 맞추어 세계관의 규칙, 배경, 주요 요소를 풍부하게 서술하세요.
    - 등장인물들의 배경, 동기, 성격 등을 자세하게 확장하세요.
    - 시놉시스의 주요 갈등과 사건을 구체적으로 설명하세요.
    - 결과는 반드시 아래 JSON 형식을 준수하여 출력하세요.
    {{
        "expanded_worldview": "확장된 세계관 내용",
        "expanded_synopsis": "확장된 시놉시스 내용",
        "expanded_characters": "확장된 등장인물 내용"
    }}

    <주어진 정보>
    소설의 장르: {genre}
    소설의 제목: {title}
    세계관 소개: {worldview}
    시놉시스: {synopsis}
    등장인물: {characters}

    **이 정보를 확장한 JSON 객체를 출력하세요.**
    """
)
# 파이프 연산자를 통해 프롬프트 템플릿과 llm을 연결하면, 체인은 callable 객체가 됩니다.
expansion_chain = expansion_prompt_template | llm

# (나) 확장된 정보를 기반으로 소설 첫 화(첫 번째 챕터)를 생성하는 프롬프트 템플릿
first_chapter_prompt_template = PromptTemplate(
    input_variables=["genre", "title", "expanded_worldview", "expanded_synopsis", "expanded_characters"],
    template="""
    <역할>
    당신은 뛰어난 소설 작가입니다. 아래 확장된 정보를 바탕으로, 몰입감 있고 감각적인 소설의 첫 화(첫 번째 챕터)를 작성하세요.

    <목표 및 작성 지침>
    - 장르는 {genre}에 맞게 설정하며, 서술은 생동감 있고 세밀하게 진행합니다.
    - 첫 화에서는 주인공 및 주요 인물들이 등장하고, 독자의 흥미를 끌 사건이나 갈등의 단서를 자연스럽게 제시합니다.
    - 한글 기준으로 공백을 제외한 문자 수가 500자 이상 700자 이하가 되도록 작성하세요. (문단 간격은 포함하지 않습니다.)

    <주어진 확장된 정보>
    소설의 장르: {genre}
    소설의 제목: {title}
    확장된 세계관: {expanded_worldview}
    확장된 시놉시스: {expanded_synopsis}
    확장된 등장인물: {expanded_characters}

    **소설 첫 화(첫 번째 챕터)를 작성하세요.**
    """
)
first_chapter_chain = first_chapter_prompt_template | llm

# (다) 이전 화의 내용과 확장된 정보를 바탕으로 후속 챕터(다음 화)를 생성하는 프롬프트 템플릿
next_chapter_prompt_template = PromptTemplate(
    input_variables=["genre", "title", "expanded_worldview", "expanded_synopsis", "expanded_characters", "previous_chapter"],
    template="""
    <역할>
    당신은 뛰어난 소설 작가입니다. 아래 확장된 정보와 이전 화의 내용을 바탕으로, 소설의 다음 화(후속 챕터)를 작성하세요.

    <목표 및 작성 지침>
    - 장르는 {genre}에 맞게 설정하며, 서술은 생동감 있고 세밀하게 진행합니다.
    - 이전 화의 내용과 자연스럽게 연결되는 전개를 보여주세요.
    - 한글 기준으로 공백을 제외한 문자 수가 500자 이상 700자 이하가 되도록 작성하세요. (문단 간격은 포함하지 않습니다.)

    <주어진 정보>
    소설의 장르: {genre}
    소설의 제목: {title}
    확장된 세계관: {expanded_worldview}
    확장된 시놉시스: {expanded_synopsis}
    확장된 등장인물: {expanded_characters}
    이전 화 내용: {previous_chapter}

    **소설 다음 화(후속 챕터)를 작성하세요.**
    """
)
next_chapter_chain = next_chapter_prompt_template | llm

# 4. 장르, 제목, 세계관, 시놉시스, 등장인물 정보를 입력받아 확장된 정보를 생성하는 함수
def generate_expanded_info(genre, title, worldview, synopsis, characters):
    # 등장인물 정보를 문자열로 변환
    characters_str = "\n".join(
        [f"- 이름: {char['name']}, 역할: {char['role']}, 나이: {char['age']}, 성별: {char['sex']}, 직업: {char['job']}, 프로필: {char['profile'].strip()}"
         for char in characters]
    )
    input_data = {
        "genre": genre,
        "title": title,
        "worldview": worldview,
        "synopsis": synopsis,
        "characters": characters_str
    }
    # 체인 호출 후 결과가 AIMessage 객체인 경우 content를 추출
    expanded_output = expansion_chain.invoke(input_data)
    if hasattr(expanded_output, "content"):
        expanded_output = expanded_output.content
    try:
        expanded_info = json.loads(expanded_output)
    except json.JSONDecodeError as e:
        raise ValueError("확장된 정보가 JSON 형식이 아닙니다: " + str(e))
    return expanded_info

# 5. 확장된 정보를 기반으로 소설 첫 화를 생성하는 함수
def generate_first_chapter(genre, title, expanded_info):
    input_data = {
        "genre": genre,
        "title": title,
        "expanded_worldview": expanded_info.get("expanded_worldview", ""),
        "expanded_synopsis": expanded_info.get("expanded_synopsis", ""),
        "expanded_characters": expanded_info.get("expanded_characters", "")
    }
    chapter = first_chapter_chain.invoke(input_data)
    if hasattr(chapter, "content"):
        chapter = chapter.content
    return chapter

# 6. 이전 소설 내용과 확장된 정보를 기반으로 소설 후속 화를 생성하는 함수
def generate_next_chapter(genre, title, expanded_info, previous_chapter):
    input_data = {
        "genre": genre,
        "title": title,
        "expanded_worldview": expanded_info.get("expanded_worldview", ""),
        "expanded_synopsis": expanded_info.get("expanded_synopsis", ""),
        "expanded_characters": expanded_info.get("expanded_characters", ""),
        "previous_chapter": previous_chapter
    }
    next_chapter = next_chapter_chain.invoke(input_data)
    if hasattr(next_chapter, "content"):
        next_chapter = next_chapter.content
    return next_chapter

if __name__ == "__main__":
    # 기본 입력 정보 설정
    genre = "SF, 디스토피아, 배틀로얄, 초능력"
    title = "이터널 리턴: 아글라이아의 실험"
    worldview = """
    지구가 형성될 때부터 존재했던 미지의 힘, **VF(Variable Force)**. 
    이 힘은 자연현상을 초월하는 강력한 능력을 부여하며, 운석을 지구로 끌어들이거나 특정 생명체에 변칙적인 힘을 발현시키는 등 신비로운 성질을 가지고 있다.

    그러나 VF의 존재는 인간에게도 미미한 영향을 끼쳤고, 그 흔적은 신화나 전설, 종교적 이야기 속에서나 남아 전해져 왔다.
    초인적인 신체 능력, 염동력, 예지력 등 전설 속에 등장하는 능력들은 실상 VF의 영향을 받은 인간들일 가능성이 컸다.
    하지만 그 횟수는 극히 적었고, 인간이 이를 과학적으로 증명하기엔 너무나도 부족한 데이터뿐이었다.

    이에 흥미를 가진 **비밀 연구 조직 '아글라이아'**는 VF의 존재를 추적하고 연구하기 시작했다.
    그들은 VF가 어떻게 발현되고, 특정한 조건에서 어떤 성질을 띄며, 인간에게 어떤 영향을 끼치는지를 실험하기 위해 2001년, 루미아 섬의 원주민들을 강제로 이주시키고,
    그곳을 **살아남기 위해 서로를 죽여야만 하는 전장으로 만들었다**.

    그러나 VF의 가능성은 끝이 없었고, 예상조차 할 수 없는 변칙성을 보여주었다.
    이에 따라, 아글라이아는 실험체의 모집 범위를 넓혀 **전 세계에서 특출난 재능을 가진 사람들, 혹은 VF 발현 가능성이 있는 인물들을 강제적으로 납치하거나 모집**하였다.

    기술력 면에서도 세계 최상급을 자랑하는 아글라이아는, 2001년 실험이 시작된 이후부터 지금까지  
    **실험체들이 늙지 않도록 조치하고, 실험이 시작될 때마다 기억을 지움으로써 실험을 '영원히 반복'하고 있다**.

    현재 진행 중인 **2차 실험**에서는, VF의 발현을 더욱 효과적으로 끌어내기 위해 실험체들에게 극한의 상황을 부여하고 있다.  
    이들은 강제적으로 폭탄 목걸이를 착용한 채 루미아 섬에서 생존해야 하며, 서로를 죽이지 않으면 금지구역 설정과 강력한 '야생동물' 투입으로 강제적으로 도태된다.  

    그러나 **일부 실험체들은 이 실험을 끝낼 방법을 찾기 위해 움직이기 시작했다...**
    """
    synopsis = """
    견습 수녀 키아라는 신의 뜻을 따라 어둠 속에서 빛을 전하려 했지만, 세상은 그녀를 외면했고 그녀의 신앙은 흔들리기 시작했다.
    잔혹한 사채업자 다르코는 폭력과 돈이 세상의 진리라 믿었고, 누구도 그의 길을 막을 수 없었다.
    이들이 강제로 끌려온 곳은 ‘아글라이아’라 불리는 정체불명의 실험 구역.

    여기서 실험체들은 서로를 죽여야만 살아남을 수 있으며, 모든 사망자는 기억을 잃고 실험이 재시작되는 ‘이터널 리턴’의 굴레에 갇혀 있다.
    이들은 단순한 생존을 넘어, 이 실험을 끝내기 위한 방법을 찾기 시작한다.

    그러나 아글라이아의 지도자 안젤리카와 급진적인 부소장 에르샤, 그리고 그들이 창조한 인류를 초월한 존재 ‘이바’와 ‘에키온’이 이 실험을 지배하고 있다.
    그들의 계획은 무엇이며, 실험체들은 이 끝없는 실험에서 벗어날 수 있을까?

    살아남기 위한 처절한 싸움 속에서, 진정한 자유를 쟁취하기 위한 전투가 벌어진다.
    키아라와 다르코는 이 실험을 끝내기 위해 함께 싸워나가며, VF의 비밀을 풀어나가기 시작한다.
    이 과정에서 키아라, 다르코 외의 실험체들을 만나고 마음이 맞는 실험체들과 함께 협력한다.
    이 이야기의 끝은 다르코의 희생으로 키아라가 VF의 비밀을 풀어내고, 실험체들이 자유롭게 풀려남으로써 끝난다.
    """
    characters = [
        {
            "name": "키아라",
            "role": "주인공",
            "age": 21,
            "sex": "여성",
            "job": "견습 수녀",
            "profile": """
            태어나자마자 폐쇄적인 성당에 맡겨졌고, 매일 하느님의 전능을 세뇌 받다시피 교육받는다.
            바깥 외출이 허가된 이후 그녀는 하느님의 사도로서 충실하기 위해 가장 어둡고 더러운 곳에서 전도를 시작했지만, 그녀가 마주한 현실은 배운 것 보다 훨씬 더 냉담했다.
            그 과정에서 충실한 사도인 자신을 지켜주지 않는 하느님을 부정하는 생각을 품게 되고, 그런 자기 자신에 대한 강한 혐오감을 가지게 된다.
            아름다운 것, 고결한 것을 보게 되면 자격지심으로 인해 폭주한다.

            폭력과 죄악을 증오하지만, 동시에 아름다움과 순수한 것에 대한 강한 집착을 가지고 있다. 
            실험 구역에서 생존을 위해 자신이 믿었던 신념과 싸우며 변화해 나간다.
            """
        },
        {
            "name": "다르코",
            "role": "조력자",
            "age": 35,
            "sex": "남성",
            "job": "사채업자",
            "profile": """
            돈과 폭력이 세상의 모든 것을 해결해준다고 믿는 남자.
            잔혹하고 무자비한 성격으로, 자신이 원하는 건 전부 빼앗아야 직성이 풀린다.
            경찰인 아버지 밑에서 자랐으나 어렸을 때부터 성격이 포악했다.
            빚에 허덕이는 아버지에게 버려질 때, 그는 속으로 환호했다.
            '드디어 이 지옥 같은 굴레에서 벗어날 수 있겠구나.'
            이후 다르코는 자신이 사랑하는 폭력을 사용해서 돈을 벌기 시작했다.
            어떤 일이든 돈과 폭력은 그에게 늘 최우선 순위가 되었고, 삶의 전부가 되었다.
            어쩌면 그가 담보라는 이름으로 팔려나갔을 때, 깨달았을지도 모른다.
            돈과 폭력이 이 세상의 전부라는 것을.

            아글라이아의 실험에 참가하게 되면서도 그는 여전히 자신의 신념을 지키려 하지만, 
            점점 이 실험이 단순한 돈과 권력을 넘어선 것임을 깨닫게 된다. 
            자신이 시스템의 일부로서 이용당하고 있음을 깨닫고, 실험을 끝낼 방법을 찾기 시작한다.
            """
        },
        {
            "name": "에르샤",
            "role": "대적자 (아글라이아 부소장)",
            "age": "40대 후반 (실제 나이 불명)",
            "sex": "여성",
            "job": "아글라이아 부소장",
            "profile": """
            아글라이아의 실험을 급진적으로 추진하는 인물. 실험체들의 한계를 시험하기 위해 직접 서울에 테러를 감행하고,
            괴물 ‘스킬라’를 풀어놓아 실험체들을 극한의 상황으로 몰아넣었다. 
            온건한 안젤리카와는 자주 대립하며, 더욱 강력한 존재를 창조하려는 목표를 가지고 있다. 
            그녀는 실험체 ‘이바’를 자신의 궁극적인 창조물로 보고 있으며, 실험의 결과를 조작하려 한다.
            """
        },
        {
            "name": "이바 (Eva)",
            "role": "불완전한 신 인류",
            "age": "불명 (신체적 나이 20대 초반)",
            "sex": "여성",
            "job": "실험체",
            "profile": """
            아글라이아가 창조한 신 인류 중 하나. 인간을 초월하는 신체 능력을 가졌으며, 
            모든 감정을 제거당한 채 실험에 참여하고 있다. 
            하지만, 실험을 거듭할수록 사라졌다고 믿었던 감정이 되살아나기 시작하며, 
            인간성과 신 인류 사이에서 갈등하게 된다. 
            자신을 ‘창조자’로 여기는 에르샤와의 관계도 점점 변화해 간다.

            말을 걸어도 상냥하게 웃어줄 뿐, 자신을 드러낼 수 있는 대답은 하지 않는다.
            그런 모습에서 어딘가 비밀스럽고 신비로운 분위기가 느껴진다.

            통각이 둔한 상태라 무언가에 잘 부딪치고 넘어진다.
            멍들고 상처가 나도 잘 모른다.
            조금만 건드려도 부서질 것 같은 가녀린 몸이지만, 모두를 두렵게 만들 강력한 힘을 숨기고 있다.
            """
        }
    ]
    
    # 4. 입력 정보를 확장하여 상세 정보를 생성
    expanded_info = generate_expanded_info(genre, title, worldview, synopsis, characters)
    print("확장된 정보:", expanded_info)
    
    # 5. 확장된 정보를 기반으로 소설 첫 화 생성
    first_chapter = generate_first_chapter(genre, title, expanded_info)
    print("\n소설 첫 화:\n", first_chapter)
    
    # 6. 첫 화를 기반으로 소설 후속 화 생성
    next_chapter = generate_next_chapter(genre, title, expanded_info, first_chapter)
    print("\n소설 다음 화:\n", next_chapter)


확장된 정보: {'expanded_worldview': "이터널 리턴: 아글라이아의 실험의 세계관은 지구 형성 이래 존재했던 미지의 힘, VF(Variable Force)를 중심으로 펼쳐진다. VF는 자연현상을 초월하는 강력한 힘으로, 운석을 지구로 끌어들이거나 특정 생명체에 변칙적인 힘을 발현시키는 능력이 있다. 이 힘은 인간의 신화, 전설, 종교적 이야기에서 간간히 찾아볼 수 있는 초능력의 원인이며, 종종 인간에게 엄청난 능력을 부여하기도 한다. 하지만, 이를 과학적으로 증명할 수 있는 데이터는 부족하다. 이를 추적하고 연구하기 위해 비밀 연구 조직 '아글라이아'가 설립되었다. 아글라이아는 루미아 섬을 무대로 인간에게 VF의 영향을 연구하는 극한의 배틀 로얄을 연출한다. 아글라이아의 연구 결과는 인간이 VF의 힘을 어떻게 활용하는지, 어떤 상황에서 더 강력한 능력을 발현하는지 등의 정보를 제공한다.", 'expanded_synopsis': "이터널 리턴: 아글라이아의 실험은 견습 수녀 키아라와 사채업자 다르코가 아글라이아의 VF 실험에 참여하게 되면서 벌어지는 이야기다. 이들은 서로 다른 이유로 이 광기의 실험에 참가하게 되었고, 각자의 신념과 가치, 그리고 인간의 본질에 대해 직면하게 된다. 실험은 서로를 죽여야만 살아남을 수 있는 극한의 상황을 연출하며, 실험체들은 이 끝이 없는 실험에서 벗어날 방법을 찾아 헤매게 된다. 하지만 아글라이아의 지도자 안젤리카와 부소장 에르샤, 그리고 그들이 창조한 인류를 초월한 존재 '이바'와 '에키온'이 실험을 지배하고 있다. 키아라와 다르코는 이 반복되는 지옥에서 벗어나기 위해, 그리고 진정한 자유를 얻기 위해 함께 싸운다. 그 과정에서 둘은 VF의 비밀을 밝혀내고, 실험이 끝나는 새로운 방법을 찾아낸다.", 'expanded_characters': '키아라는 견습 수녀로, 신의 뜻을 따라 어둠 속에서 빛을 전하려 했다. 그러나 세상은 그녀를 외면하고 그녀의 신앙은 흔들렸다. 그녀는 아름다움과 순수함에 대한 강한 집착

In [18]:
print(next_chapter)

"이터널 리턴: 아글라이아의 실험" 두 번째 챕터 - VF의 깨어남

불안감과 기대가 얽히는 가운데, 루미아 섬의 중심지에서 비명 같은 경고음이 울려 퍼졌다. 따라잡을 수 없는 순간, 스멀스멀하게 떠오르는 VF의 힘이 강력하게 퍼져나갔다. 키아라와 다르코, 그리고 이바는 각자의 위치에서 이 변화를 느꼈다.

키아라는 신의 빛을 느꼈다고 생각했다. "신이 주신 힘이라면, 이를 통해 세상을 바로잡을 수 있을 것이다." 그녀는 강렬한 빛을 받아들이며, 자신의 VF 능력, '빛의 노래'가 깨어났다. 그녀의 손에서는 밝은 빛이 터져 나오며, 주변의 모든 것을 밝게 비추었다.

다르코는 이를 더욱 강렬하게 느꼈다. "이것이 그들이 말하는 능력인가?" 그의 팔에서는 검은 불꽃이 불타오르며, 그의 VF 능력, '파멸의 힘'이 깨어났다. 그의 눈은 불꽃에 휩싸인 것처럼 붉게 빛났다.

한편 에르샤는 실험실에서 이들의 변화를 지켜보았다. "기대 이상이군. 이들의 능력이 어떻게 발현될지 기대된다." 그의 눈에는 차가운 빛이 스멀스멀 떠올랐다.

그러나, 가장 큰 변화는 이바에게서 일어났다. 그의 몸에서는 신비한 파란 빛이 퍼져나가며, 그의 VF 능력, '감정의 파도'가 깨어났다. 그의 눈은 빛과 어둠, 평온함과 혼란이 교차하는 듯한 모습을 보였다.

"감정이... 다시 느껴지는건가?" 그는 이상한 기분에 휩싸여, 자신의 감정에 대한 이해를 찾아 헤맴했다.

이처럼, VF의 힘이 깨어난 이들은 자신들의 능력을 이해하고, 이를 활용하는 방법을 배우기 시작했다. 이들의 배틀로얄은 더욱 치열해지며, 아글라이아의 실험은 한 걸음 더 깊이 들어갔다. 이상과 현실, 그리고 감정과 능력이 충돌하는 이 곳에서, 이들은 자신의 길을 찾아야만 했다.
